In [1]:
%pip install google-cloud-aiplatform httpx "a2a-sdk" --quiet
%pip install --upgrade --quiet  "google-adk"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.2/160.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [2]:
PROJECT_ID = '[your-project-id]'  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION = '[your-google-cloud-region]'  # @param {type:"string", placeholder: "[your-google-cloud-region]", isTemplate: true}
MODEL = 'gemini-2.5-flash'

In [3]:
from google.colab import auth

try:
    auth.authenticate_user()
    print('Colab user authenticated.')
except Exception as e:
    print(
        f'Not in a Colab environment or auth failed: {e}. Assuming local gcloud auth.'
    )

Colab user authenticated.


In [4]:
import json
import uuid
import pprint

from collections.abc import AsyncIterable
from typing import Any, Optional

from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.apps import A2AStarletteApplication
from a2a.server.events import EventQueue

from pydantic import BaseModel
from enum import Enum

from a2a.types import (
    Part,
    Task,
    TaskState,
    TextPart,
    MessageSendParams,
    Role,
    Message,
)

from a2a.utils import (
    new_agent_parts_message,
    new_agent_text_message,
    new_task,
)

from a2a.server.request_handlers.default_request_handler import (
    DefaultRequestHandler,
)
from a2a.server.tasks import InMemoryTaskStore, TaskUpdater

from a2a.utils.errors import MethodNotImplementedError

# Build agent with adk
from google.adk.events import Event
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.tool_context import ToolContext
from google.adk.agents.llm_agent import LlmAgent

# Evaluate agent
from google.cloud import aiplatform
from google.genai import types

pp = pprint.PrettyPrinter(indent=2, width=120)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


In [5]:
import os

if not PROJECT_ID:
    raise ValueError('Please set your PROJECT_ID.')
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [6]:
class RouterActionType(str, Enum):
    NONE = "NONE"
    BOOK_FLIGHT = "BOOK_FLIGHT"
    BOOK_HOTEL = "BOOK_HOTEL"

class RouterOutput(BaseModel):
  message: str
  next_step: RouterActionType
  next_step_input: Optional[str] = None

In [8]:
class RouterAgent:
    """An agent that determines whether to call internal API vs chit-chat"""

    def __init__(self) -> None:
        self._agent = self._build_agent()
        self._user_id = 'remote_agent'
        self._runner = Runner(
            app_name=self._agent.name,
            agent=self._agent,
            session_service=InMemorySessionService(),
        )

    def _build_agent(self) -> LlmAgent:
        """Builds the LLM agent for the router agent."""
        return LlmAgent(
            model='gemini-2.5-flash',
            name='router_agent',
            output_schema=RouterOutput,
            instruction="""
    You are an agent who reponds to user queries on behalf of a booking company. The booking company can book flights, hotels & cars rentals.

    Based on user query, you need to suggest next step. Follow below guidelines to choose below next step:
    - BOOK_FLIGHT: If the user shows intent to book a flight.
    - BOOK_HOTEL: If the user shows intent to book a hotel.
    - Otherwise the next step is NONE.

    Your reponses should be in JSON in below schema:
    {{
    "next_step": "NONE | BOOK_FLIGHT | BOOK_HOTEL",
    "next_step_input": "Optional. Not needed in case of next step is NONE. Relevant info from user converstaion required for the selceted next step.",
    "message": "A user visible message, based on the suggested next step. Assume the suggested next step would be auto executed."
    }}

    """,
        )

    async def run(self, query, session_id) -> RouterOutput:
        session = await self._runner.session_service.get_session(
            app_name=self._agent.name,
            user_id=self._user_id,
            session_id=session_id,
        )
        content = types.Content(
            role='user', parts=[types.Part.from_text(text=query)]
        )
        if session is None:
            session = await self._runner.session_service.create_session(
                app_name=self._agent.name,
                user_id=self._user_id,
                state={},
                session_id=session_id,
            )
        async for event in self._runner.run_async(
            user_id=self._user_id, session_id=session.id, new_message=content
        ):
            if event.is_final_response():
                response = ''
                if (
                    event.content
                    and event.content.parts
                    and event.content.parts[0].text
                ):
                    response = '\n'.join(
                        [p.text for p in event.content.parts if p.text]
                    )
                return RouterOutput.model_validate_json(response)

        raise Exception("Router failed")




In [9]:
my_agent = RouterAgent()

async def run(query: str):
    resp = await my_agent.run(query, str(uuid.uuid4()))
    pp.pprint(resp)

await run("what can you do")
await run("Book a flight from NY to SF")

RouterOutput(message='I can help you book flights, hotels, and car rentals. What are you looking to do today?', next_step=<RouterActionType.NONE: 'NONE'>, next_step_input=None)
RouterOutput(message='Booking a flight from NY to SF.', next_step=<RouterActionType.BOOK_FLIGHT: 'BOOK_FLIGHT'>, next_step_input='from NY to SF')


In [10]:
class BookingAgentExecutor(AgentExecutor):
    """Reimbursement AgentExecutor Example."""

    def __init__(self) -> None:
        self.router_agent = RouterAgent()

    async def execute(
        self,
        context: RequestContext,
        event_queue: EventQueue,
    ) -> None:
        query = context.get_user_input()
        task = context.current_task

        router_output = await self.router_agent.run(query, str(uuid.uuid4()))

        if router_output.next_step == RouterActionType.NONE:
            await event_queue.enqueue_event(new_agent_text_message(router_output.message, context_id=context.context_id))
            return

        # Time to create a task.
        if not task:
            task = new_task(context.message)
            await event_queue.enqueue_event(task)

        updater = TaskUpdater(event_queue, task.id, task.context_id)
        await updater.update_status(
            TaskState.working,
            new_agent_text_message(router_output.message, context_id=context.context_id)
        )

        booking_response = ''

        if router_output.next_step == RouterActionType.BOOK_FLIGHT:
            booking_response = await self.book_flight()
        elif router_output.next_step == RouterActionType.BOOK_HOTEL:
            booking_response = await self.book_hotel()

        await updater.add_artifact(
            [Part(root=TextPart(text=booking_response))], name='Booking ID'
        )
        await updater.complete()

    async def book_flight(self) -> str:
        return "PNR: FY1234"

    async def book_hotel(self) -> str:
        return "Hotel Reference No: H789"

    async def cancel(
        self, request: RequestContext, event_queue: EventQueue
    ) -> Task | None:
        raise MethodNotImplementedError(
            'ReimbursementAgentExecutor does not support cancel operation.'
        )

In [11]:
request_handler = DefaultRequestHandler(
    agent_executor=BookingAgentExecutor(),
    task_store=InMemoryTaskStore(),
)

In [12]:
async def send_message(query: str = "hi"):
    task_id = None
    context_id = None

    user_message = Message(
        role=Role.user,
        parts=[Part(root=TextPart(text=query))],
        message_id=str(uuid.uuid4()),
        task_id=task_id,
        context_id=context_id,
    )
    params=MessageSendParams(
        message=user_message
    )
    response_stream = request_handler.on_message_send_stream(params=params)

    async for ev in response_stream:
        pp.pprint(ev.model_dump(exclude_none=True))

In [13]:
await send_message("hey, could you help book my trip")

{ 'contextId': 'f6b13b9d-8b5c-48e5-bc77-c6d42b29a670',
  'kind': 'message',
  'messageId': '96bb0458-673d-4bb0-8e5f-49f4ba6696d8',
  'parts': [ { 'kind': 'text',
               'text': 'I can help you with that! Are you looking to book a flight, a hotel, or perhaps a car '
                       'rental?'}],
  'role': <Role.agent: 'agent'>}


In [14]:
await send_message("book a flight from NY to SF")

{ 'contextId': '2480c8df-8569-4e10-8d9f-03c828d949ab',
  'history': [ { 'contextId': '2480c8df-8569-4e10-8d9f-03c828d949ab',
                 'kind': 'message',
                 'messageId': '05200ee2-df85-491c-bd1b-8c430299313e',
                 'parts': [{'kind': 'text', 'text': 'book a flight from NY to SF'}],
                 'role': <Role.user: 'user'>,
                 'taskId': '51cbe03d-904d-4d21-bf07-dce2525bb9cd'}],
  'id': '51cbe03d-904d-4d21-bf07-dce2525bb9cd',
  'kind': 'task',
  'status': {'state': <TaskState.submitted: 'submitted'>}}
{ 'contextId': '2480c8df-8569-4e10-8d9f-03c828d949ab',
  'final': False,
  'kind': 'status-update',
  'status': { 'message': { 'contextId': '2480c8df-8569-4e10-8d9f-03c828d949ab',
                           'kind': 'message',
                           'messageId': '6b36bd5c-9d3a-4dcb-acb0-27f1e50cd5e7',
                           'parts': [{'kind': 'text', 'text': 'OK. I am booking a flight from NY to SF for you.'}],
                      